# Rescatar datos y soluciones

## Modelo

In [2]:
# Tarea 2 Operacion Economica de Sistemas Electricos
# Vicente Goehring & Sebastian Villagra
# Mayo 2024

# Codigo busca crear un modelo de optimizacion para el Unit Commitment
# de forma mas general posible, i.e., entra como input cualquier CSV con
# los datos de un sistema y optimiza de forma robusta.


#Iniciación de paquetes
using DataFrames, CSV
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi
using XLSX  # Se agrega para leer archivos .xlsx (Excel)

#Definición de struct: FALTA AGREGAR MAS INSTANCIAS DE CADA STRUCT (seba)  

# Barras
struct Bus
    Id::Int64       # Id del Bus
    Vmax::Float64   # Voltaje maximo [pu]
    Vmin::Float64   # Voltaje minimo [pu]
    Gs::Float64
    Bs::Float64
end

# Generadores
struct Generador
    Name::String
    Bus::Int64    
    PotMax::Float64
    PotMin::Float64
    Qmax::Float64
    Qmin::Float64
    Ramp::Float64
    Sramp::Float64
    MinUp::Float64
    MinDn::Float64
    InitS::Float64
    InitP::Float64
    StartCost::Float64
    FixedCost::Float64
    VariableCost::Float64
    Tech::String
    PminFactor::Float64 
    Qfactor::Float64
    RampFactor::Float64
    StartUpCostFactor::Int64
end

# Potencias demandadas
# La demanda la dejaria como un DataFrame y listo
struct Demanda
    T::Int64
    Barra1::Int64
    Barra2::Int64
    Barra3::Int64
    Barra4::Int64
    Barra5::Int64
    Barra6::Int64
    Barra7::Int64
    Barra8::Int64
    Barra9::Int64
end

# Lineas de transmision
# IMPORTANTE:   case014.xlsx tiene el inicio y fin como string. case118.xlsx los tiene como int
#               Por lo tanto, se modifica el archivo case014.xlsx para poder leer de forma automatica
struct Linea
    Name::String    
    Inicio::Int64       # Barra de Inicio
    Fin::Int64          # Barra de Fin
    R::Float64          # Resistencia
    X::Float64          # Reactancia
    LineCharging::Int64 # (B)
    PotMax::Int64       # Potencia maxima a transportar
end
# Battery Energy Storage System
struct BESS
    Id::Int64
    PotMax::Int64
    Duration::Int64
    Eff::Float64
    Estart::Float64
    Efinal::Float64
    Barra::Int64
end
# Renewables -> tambien dejarlo como DataFrame

##  Archivo Excel a leer
archivo = "Case014.xlsx"    # se comienza con el caso mas pequeño
xf = XLSX.readxlsx(archivo) # leer el archivo excel

# Separar hojas del archivo
sheet_buses = xf["Buses"]
sheet_demandas = xf["Demand"]
sheet_generadores = xf["Generators"]
sheet_lineas = xf["Lines"]
sheet_renovables = xf["Renewables"]

# Definir la función de parada
# Se crea el DataFrame hasta que se reconozca la entrada "END"
# De esta forma se puede leer tablas con cualquier numero de filas
#   siempre y cuando terminen con un "END".
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

# Crear DataFrames
# XLSX.gettable(objeto hoja excel, Columnas de tabla, first_row= primera fila a considerar, stop_in_row_function: condicion dejar de leer)
buses_ref       = DataFrame(XLSX.gettable(sheet_buses,"A:E",first_row=1,stop_in_row_function=stop_condition))
demandaP_ref    = DataFrame(XLSX.gettable(sheet_demandas,"A:Y",first_row=2,stop_in_row_function=stop_condition))
demandaQ_ref    = DataFrame(XLSX.gettable(sheet_demandas,"AA:AY",first_row=2,stop_in_row_function=stop_condition))
generadores_ref = DataFrame(XLSX.gettable(sheet_generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))
lineas_ref      = DataFrame(XLSX.gettable(sheet_lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))
renovables_ref  = DataFrame(XLSX.gettable(sheet_renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))
# Notar que se crean 2 DataFrame a partir de la hoja "Demand": Potencia activa (P) y potencia reactiva (Q).
# Ademas, la fila considerada como inicio de la tabla es la fila 2, ya que la fila 1 solo hace distincion entre estas 2 potencias.
# Lo mismo ocurre en el DataFrame "renovables_ref".

# ESTO SE ELIMINARIA
#######################################################
#Importación de archivos
#generators_ref  = CSV.File("Generators.csv") |> DataFrame   # Crear DataFrame generadores
#demand_ref      = CSV.File("Demand.csv") |> DataFrame       # Crear DataFrame demanda
#lines_ref       = CSV.File("Lines.csv") |> DataFrame        # Crear DataFrame lineas tx
#batteries_ref   = CSV.File("Bess.csv") |> DataFrame         # Crear DataFrame baterias
# Crear una copia 
#generators      = copy(generators_ref)
#demand          = copy(demand_ref)
#lines           = copy(lines_ref)
#batteries       = copy(batteries_ref)
#######################################################

# Crear copias (aparentemente buena practica)
buses = copy(buses_ref)
demandP = copy(demandaP_ref[:,2:25])
demandQ = copy(demandaQ_ref)
generators = copy(generadores_ref)
lines = copy(lineas_ref)
renewables = copy(renovables_ref) 

#inclusión en variables
P_base = 100
T = ncol(demandP)           # N° bloques temporales
N = nrow(buses)             # N° nodos 
I = nrow(generators)        # N° Generadores
L = nrow(lines)             # N° Lineas
R = nrow(renewables)        # N° Renovables
#B = nrow(batteries)        # N° BESS

# Debugg check
println("Prueba de numero buses, deberia ser 14: ", N)

# HASTA ACA LLEGUE (SEA -quien mas xdxdxdxd): 17/05 -

# Listas que almacenan los Structs 
Buses       = []
Gen         = []
Demandas    = []
Lineas      = []
#Baterias    = []

## Crear los Structs

# Struct Bus
# struct Bus
#     Id::Int64       # Id del Bus
#     Vmax::Float64   # Voltaje maximo [pu]
#     Vmin::Float64   # Voltaje minimo [pu]
#     Gs::Float64
#     Bs::Float64
# end
for i in 1:N
    x = Bus(
        buses[i,1], # 1- Id
        buses[i,2], # 2- Vmax [pu]
        buses[i,3], # 3- Vmin [pu]
        buses[i,4], # 4- Gs
        buses[i,5]) # 8- Bs   
    push!(Buses, x)
end

# Struct Generadores
# struct Gen
#     (1)Id:: Int64              
#     (2)Name::String
#     (3)Bus::Int64    
#     (4)PotMax::Float64
#     (5)PotMin::Float64
#     (6)Qmax::Float64
#     (7)Qmin::Float64
#     (8)Ramp::Int64
#     (9)Sramp::Int64
#     (10)MinUp::Int64
#     (11)MinDn::Int64
#     (12)InitS::Int64
#     (13)InitP::Int64
#     (14)StartCost::Int64
#     (15)FixedCost::Int64
#     (16)VariableCost::Int64
#     (17)Type::String
#     (18)PminFactor::Float64 
#     (19)Qfactor::Float64
#     (20)RampFactor::Float64
#     (21)StartUpCostFactor::Int64
# end

for i in 1:I
    println("Generador", i)
    x = Generador(
        generators[i,1],    # 1- Name
        generators[i,2],    # 2- Bus
        generators[i,3],    # 3- Pmax
        generators[i,4],    # 4- Pmin
        generators[i,5],    # 5- Qmax
        generators[i,6],    # 6- Qmin
        generators[i,7],    # 7- Ramp [Mw/h]
        generators[i,8],    # 8- Sramp [Mw]
        generators[i,9],    # 9- MinUp
        generators[i,10],   # 10- MinDn
        generators[i,11],   # 11- InitS
        generators[i,12],   # 12- InitP
        generators[i,13],   # 13- StartCost
        generators[i,14],   # 14- FixedCost
        generators[i,15],   # 15- VariableCost
        generators[i,16],   # 16- Type
        generators[i,17],   # 17- PminFactor
        generators[i,18],   # 18- Qfactor
        generators[i,19],   # 19- RampFactor
        generators[i,20])   # 20- StartUpCostFactor
    push!(Gen, x)
end
## println(Generadores[1].PotMax) ejemplo de como obtener un prámetro en particular

# Struct Linea
# struct Linea
#     Id::Int64
#     Name::String    
#     Inicio::Int64       # Barra de Inicio
#     Fin::Int64          # Barra de Fin
#     R::Float64          # Resistencia
#     X::Float64          # Reactancia
#     LineCharging::Int64 # (B)
#     PotMax::Int64       # Potencia maxima a transportar
# end
for i in 1:L
    x = Linea(
        lines[i,1], # 1- Name 
        lines[i,2], # 2- StartBus
        lines[i,3], # 3- EndBus
        lines[i,4], # 4- Resistance
        lines[i,5], # 5- Reactance
        lines[i,6], # 6- LineCharging(B)
        lines[i,7]) # 7- MaxFlow [Mw]   
    push!(Lineas, x)
end

println("Demanda nodo 14, hora 2: ", demandP[14,2])
println("Size de demandaP_ref: ", size(demandaP_ref))
println("Size de demandP: ", size(demandP))

# Struct demanda (quizas quitarlo)
# for i in 1:N
#     x = [demand[i,2],demand[i,3],demand[i,4],demand[i,5],demand[i,6],demand[i,7]]
#     push!(Demandas, x)
# end

# Crear Structs BESS
# for b in 1:B
#     x = BESS(batteries[b,1],batteries[b,2],batteries[b,3],batteries[b,4],
#             batteries[b,5],batteries[b,6],batteries[b,7])
#     push!(Baterias, x)
# end


####################################################################
#               MODELO OPTIMIZACION
####################################################################

### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables

# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
@objective(model, Min, sum(Gen[i].VariableCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

### Restricciones

# Equilibrio de Potenica (APROXIMACION: Flujo DC  --> Demanda Potencia Activa)
@constraint(model, DCPowerFlowConstraint[n in 1:N, t in 1:T], 
sum(p[i,t] for i in 1:I if Gen[i].Bus == n) - demandP[n,t]      == 
P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) for l in 1:L if Lineas[l].Inicio == n)
+ P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Fin,t] - d[Lineas[l].Inicio,t]) for l in 1:L if Lineas[l].Fin == n))

# + sum(pb[b,t] for b in 1:B if Baterias[b].Barra == n) | Al lado izquierdo de la ecuación (baterias)

#Flujo en lineas. Se considera o de origen, y d de destino
@constraint(model, LineMaxPotInicioFin[l in 1:L, t in 1:T], 1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base) 
@constraint(model, LineMinPotFinInicio[l in 1:L, t in 1:T], -1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base)
#Angulo de referencia
@constraint(model, RefDeg[1, t in 1:T], d[1,t] == 0)  

### Restricciones de generadores

## Potencias Activas y reactivas
# Potencia Activa maxima
@constraint(model, PMaxConstraint[i in 1:I, t in 1:T], p[i,t] <= w[i,t]*Gen[i].PotMax)
# Potencia Activa minima
@constraint(model, PMinConstraint[i in 1:I, t in 1:T], w[i,t]*Gen[i].PotMin <= p[i,t])
# Potencia Renewables (condicionado a meteorologia)
@constraint(model, RenewableMax[i in (I-R+1):I, t in 1:T], p[i,t] <= renewables[i-(I-R),t+1])

## Rampas
# Rampa up
@constraint(model, RampUpConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) <= Gen[i].Ramp + Gen[i].Sramp*u[i,t])
# Rampa dn
@constraint(model, RampDownConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) >= 0-Gen[i].Ramp - Gen[i].Sramp*v[i,t])

## Estados Binarios
@constraint(model, BinaryState[i in 1:I, t in 2:T], (u[i,t] - v[i,t]) == (w[i,t] - w[i,t-1]))
#@constraint(model, BinaryInitial[i in 1:I], (w[i,1] = State0[i]))   # Condicion inicial, por ahora libre

## Tiempo minimo de encendido: sumo todos los x dentro de la ventana desde t=1 hasta el instante enterior al encendido
@constraint(model, MinUpTime[i in 1:I, t in 2:T], sum(w[i,k] for k in 1:(t-1) if k >= t-Gen[i].MinUp) >= v[i,t]*Gen[i].MinUp)
## Tiempo minimo de apagado
@constraint(model, MinDnTime[i in 1:I, t in 2:T], sum((1-w[i,k]) for k in 1:(t-1) if k >= t-Gen[i].MinDn) >= u[i,t]*Gen[i].MinDn)



# Restricciones de BESS
# Energia maxima
#@constraint(model, EnergyMax[b in 1:B, t in 1:T], e[b,t] <= (Baterias[b].PotMax)*(Baterias[b].Duration))
# Potencia maxima descarga
#@constraint(model, PMaxBessDischarge[b in 1:B, t in 1:T], pb[b,t] <= Baterias[b].PotMax)
# Potencia maxima carga
#@constraint(model, PMaxBessCharge[b in 1:B, t in 1:T], pb[b,t] >= -1*(Baterias[b].PotMax))
## Imposibilidad de generacion - E inicial
#@constraint(model, Estart[b in 1:B], e[b,1] == (Baterias[b].Estart)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Imposibilidad de generacion - E final
#@constraint(model, Efinal[b in 1:B], e[b,T] == (Baterias[b].Efinal)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Dinamica BESS
#@constraint(model, DynamicBESS[b in 1:B, t in 2:T], e[b,t] == e[b,t-1] - pb[b,t])

#############################################################


################################################ ACTUALIZADO 24-05 (00:10 AM) -SEA ################################
### Notas:
#   - Siento que hay que agregar BinaryState0 -> Con los resultados se pueden analizar incongruencias
#   - Por lo mismo, las constraints de MinUpTime y MinDnTime no me terminan de convencer
#   - SI O SI, hay que fijar p[r,t] == gr[t], es decir, las potencias generadas por renovables estan
#       fijas. HINT: los renovables están al final de la tabla de generadores. 

# constraint[i in (I-a):I, t in 1:T], p[i,t] == renewable[i,t] | una wea asi

##############################################################


JuMP.optimize!(model)

Prueba de numero buses, deberia ser 14: 14
Generador1
Generador2
Generador3
Generador4
Generador5
Generador6
Generador7
Demanda nodo 14, hora 2: 12.150115830115833
Size de demandaP_ref: (14, 25)
Size de demandP: (14, 24)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2509 rows, 1008 columns and 7104 nonzeros
Model fingerprint: 0x01d8fcf5
Variable types: 504 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [2e+01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 4e+02]
Presolve removed 1529 rows and 395 columns
Presolve time: 0.08s
Presolved: 980 rows, 613 columns, 3911 nonzeros
Variabl

┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers C:\Users\sevil\.julia\packages\JuMP\OUdu2\src\Containers\DenseAxisArray.jl:186


## Resumen

In [74]:
solution_summary(model)

* Solver : Gurobi

* Status
  Result count       : 6
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 2.11293e+05
  Objective bound    : 2.11293e+05
  Relative gap       : 0.00000e+00
  Dual objective value : 2.11293e+05

* Work counters
  Solve time (sec)   : 4.43000e-01
  Barrier iterations : 0
  Node count         : 1


### Costos

In [3]:
totalCost = objective_value(model)
fixCost = zeros(Float64, (I,T))     # fijo
varCost = zeros(Float64, (I,T))     # variable
startCost = zeros(Float64, (I,T))   # encendido

# Recorrer
for i = 1:I                                                 # Recorrer filas    (generadores)
    for t = 1:T                                             # Recorrer columnas (horas)
        fixCost[i,t] = value(w[i,t])*Gen[i].FixedCost
        varCost[i,t] = value(p[i,t])*Gen[i].VariableCost
        startCost[i,t] = value(u[i,t])*Gen[i].StartCost
    end
end

fixCostT = zeros(Float64, (1,T))
varCostT = zeros(Float64, (1,T))
startCostT = zeros(Float64, (1,T))

for t=1:T
    fixCostT[1,t] = sum(fixCost[:,t])
    varCostT[1,t] = sum(varCost[:,t])
    startCostT[1,t] = sum(startCost[:,t])
end

In [133]:
startCostT

1×24 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  2250.0  0.0  0.0  0.0  0.0  0.0

### Capacidad nominal por hora
Cuenta si los generadores están encendidos o no

In [4]:
nomCap = zeros(Float64, (I,T))
for t=1:T
    for i=1:(I-R)   # Generadores convencionales
        nomCap[i,t] = value(w[i,t])*Gen[i].PotMax
    end
    for i=(I-R+1):I # Generadores ERNC
        nomCap[i,t] = value(w[i,t])*renewables[i-(I-R),t+1]
    end
end
nomCap

7×24 Matrix{Float64}:
 100.0    100.0     100.0     100.0     …  100.0    100.0     100.0
  75.0     75.0      75.0      75.0         75.0     75.0      75.0
  75.0     75.0      75.0      75.0         75.0     75.0      75.0
   0.0      0.0       0.0       0.0         75.0      0.0       0.0
   0.0      0.0       0.0       0.0          0.0      0.0       0.0
  31.167   29.4256   27.6723   28.6323  …   35.471   32.5038   31.2067
   0.0      0.0       0.0       0.0          0.0      0.0       0.0

In [108]:
value(w[6,1])
renewables[6-(I-R),1]

"Wind2"

## Exportar resultados XLSX

Se guardan las matrices de las variables optimizadas en variables

In [5]:
v_p = JuMP.value.(p)
v_u = JuMP.value.(u)
v_v = JuMP.value.(v)
v_w = JuMP.value.(w)

7×24 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  0.0  1.0  0.0  1.0  0.0

Se crea un archivo XLSX con una hoja para cada solución

In [6]:
XLSX.openxlsx("Resultados14.xlsx", mode="w") do xf
    
    # Cambiar nombre primera hoja ("Sheet1")
    sheet1 = xf[1]
    println(sheet1)
    XLSX.rename!(sheet1, "Potencias")

    # Agregar Hojas
    XLSX.addsheet!(xf, "Encendido")
    XLSX.addsheet!(xf, "Apagado")
    XLSX.addsheet!(xf, "State")
    XLSX.addsheet!(xf, "Costos")
    XLSX.addsheet!(xf, "PotNominal")

    # Asignar variables a cada hoja
    sheet1 = xf[1]   # Hoja 1 -> Potencias
    sheet2 = xf[2]   # Hoja 2 -> Endendidos
    sheet3 = xf[3]   # Hoja 3 -> Apagados
    sheet4 = xf[4]   # Hoja 4 -> State On/Off
    sheet5 = xf[5]   # Hoja 5 -> Costos
    sheet6 = xf[6]   # Hoja 6 -> Potencial de generacion

    # Añadir las matrices
    sheet1["A1"] = v_p
    sheet2["A1"] = v_u
    sheet3["A1"] = v_v
    sheet4["A1"] = v_w
    # Hoja costos: meter 3 vectores
    sheet5["A1"] = "Costo Encendido"
    sheet5["B1"] = startCostT
    sheet5["A2"] = "Costo Fijo"
    sheet5["B2"] = fixCostT
    sheet5["A3"] = "Costo Variable"
    sheet5["B3"] = varCostT
    # Potencial
    sheet6["A1"] = nomCap
end

1×1 XLSX.Worksheet: ["Sheet1"](A1:A1) 


7×24 Matrix{Float64}:
 100.0    100.0     100.0     100.0     …  100.0    100.0     100.0
  75.0     75.0      75.0      75.0         75.0     75.0      75.0
  75.0     75.0      75.0      75.0         75.0     75.0      75.0
   0.0      0.0       0.0       0.0         75.0      0.0       0.0
   0.0      0.0       0.0       0.0          0.0      0.0       0.0
  31.167   29.4256   27.6723   28.6323  …   35.471   32.5038   31.2067
   0.0      0.0       0.0       0.0          0.0      0.0       0.0

# Modelo relajado
Se comentan las 2 restricciones de tiempos mínimos de On/Off. Se juntan todas las celdas anteriores para rescatar los valores de forma directa.

In [7]:
# Tarea 2 Operacion Economica de Sistemas Electricos
# Vicente Goehring & Sebastian Villagra
# Mayo 2024

# Codigo busca crear un modelo de optimizacion para el Unit Commitment
# de forma mas general posible, i.e., entra como input cualquier CSV con
# los datos de un sistema y optimiza de forma robusta.


#Iniciación de paquetes
using DataFrames, CSV
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi
using XLSX  # Se agrega para leer archivos .xlsx (Excel)

#Definición de struct: FALTA AGREGAR MAS INSTANCIAS DE CADA STRUCT (seba)  

# Barras
struct Bus
    Id::Int64       # Id del Bus
    Vmax::Float64   # Voltaje maximo [pu]
    Vmin::Float64   # Voltaje minimo [pu]
    Gs::Float64
    Bs::Float64
end

# Generadores
struct Generador
    Name::String
    Bus::Int64    
    PotMax::Float64
    PotMin::Float64
    Qmax::Float64
    Qmin::Float64
    Ramp::Float64
    Sramp::Float64
    MinUp::Float64
    MinDn::Float64
    InitS::Float64
    InitP::Float64
    StartCost::Float64
    FixedCost::Float64
    VariableCost::Float64
    Tech::String
    PminFactor::Float64 
    Qfactor::Float64
    RampFactor::Float64
    StartUpCostFactor::Int64
end

# Potencias demandadas
# La demanda la dejaria como un DataFrame y listo
struct Demanda
    T::Int64
    Barra1::Int64
    Barra2::Int64
    Barra3::Int64
    Barra4::Int64
    Barra5::Int64
    Barra6::Int64
    Barra7::Int64
    Barra8::Int64
    Barra9::Int64
end

# Lineas de transmision
# IMPORTANTE:   case014.xlsx tiene el inicio y fin como string. case118.xlsx los tiene como int
#               Por lo tanto, se modifica el archivo case014.xlsx para poder leer de forma automatica
struct Linea
    Name::String    
    Inicio::Int64       # Barra de Inicio
    Fin::Int64          # Barra de Fin
    R::Float64          # Resistencia
    X::Float64          # Reactancia
    LineCharging::Int64 # (B)
    PotMax::Int64       # Potencia maxima a transportar
end
# Battery Energy Storage System
struct BESS
    Id::Int64
    PotMax::Int64
    Duration::Int64
    Eff::Float64
    Estart::Float64
    Efinal::Float64
    Barra::Int64
end
# Renewables -> tambien dejarlo como DataFrame

##  Archivo Excel a leer
archivo = "Case014.xlsx"    # se comienza con el caso mas pequeño
xf = XLSX.readxlsx(archivo) # leer el archivo excel

# Separar hojas del archivo
sheet_buses = xf["Buses"]
sheet_demandas = xf["Demand"]
sheet_generadores = xf["Generators"]
sheet_lineas = xf["Lines"]
sheet_renovables = xf["Renewables"]

# Definir la función de parada
# Se crea el DataFrame hasta que se reconozca la entrada "END"
# De esta forma se puede leer tablas con cualquier numero de filas
#   siempre y cuando terminen con un "END".
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

# Crear DataFrames
# XLSX.gettable(objeto hoja excel, Columnas de tabla, first_row= primera fila a considerar, stop_in_row_function: condicion dejar de leer)
buses_ref       = DataFrame(XLSX.gettable(sheet_buses,"A:E",first_row=1,stop_in_row_function=stop_condition))
demandaP_ref    = DataFrame(XLSX.gettable(sheet_demandas,"A:Y",first_row=2,stop_in_row_function=stop_condition))
demandaQ_ref    = DataFrame(XLSX.gettable(sheet_demandas,"AA:AY",first_row=2,stop_in_row_function=stop_condition))
generadores_ref = DataFrame(XLSX.gettable(sheet_generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))
lineas_ref      = DataFrame(XLSX.gettable(sheet_lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))
renovables_ref  = DataFrame(XLSX.gettable(sheet_renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))
# Notar que se crean 2 DataFrame a partir de la hoja "Demand": Potencia activa (P) y potencia reactiva (Q).
# Ademas, la fila considerada como inicio de la tabla es la fila 2, ya que la fila 1 solo hace distincion entre estas 2 potencias.
# Lo mismo ocurre en el DataFrame "renovables_ref".

# ESTO SE ELIMINARIA
#######################################################
#Importación de archivos
#generators_ref  = CSV.File("Generators.csv") |> DataFrame   # Crear DataFrame generadores
#demand_ref      = CSV.File("Demand.csv") |> DataFrame       # Crear DataFrame demanda
#lines_ref       = CSV.File("Lines.csv") |> DataFrame        # Crear DataFrame lineas tx
#batteries_ref   = CSV.File("Bess.csv") |> DataFrame         # Crear DataFrame baterias
# Crear una copia 
#generators      = copy(generators_ref)
#demand          = copy(demand_ref)
#lines           = copy(lines_ref)
#batteries       = copy(batteries_ref)
#######################################################

# Crear copias (aparentemente buena practica)
buses = copy(buses_ref)
demandP = copy(demandaP_ref[:,2:25])
demandQ = copy(demandaQ_ref)
generators = copy(generadores_ref)
lines = copy(lineas_ref)
renewables = copy(renovables_ref) 

#inclusión en variables
P_base = 100
T = ncol(demandP)           # N° bloques temporales
N = nrow(buses)             # N° nodos 
I = nrow(generators)        # N° Generadores
L = nrow(lines)             # N° Lineas
R = nrow(renewables)        # N° Renovables
#B = nrow(batteries)        # N° BESS

# Debugg check
println("Prueba de numero buses, deberia ser 14: ", N)

# HASTA ACA LLEGUE (SEA -quien mas xdxdxdxd): 17/05 -

# Listas que almacenan los Structs 
Buses       = []
Gen         = []
Demandas    = []
Lineas      = []
#Baterias    = []

## Crear los Structs

# Struct Bus
# struct Bus
#     Id::Int64       # Id del Bus
#     Vmax::Float64   # Voltaje maximo [pu]
#     Vmin::Float64   # Voltaje minimo [pu]
#     Gs::Float64
#     Bs::Float64
# end
for i in 1:N
    x = Bus(
        buses[i,1], # 1- Id
        buses[i,2], # 2- Vmax [pu]
        buses[i,3], # 3- Vmin [pu]
        buses[i,4], # 4- Gs
        buses[i,5]) # 8- Bs   
    push!(Buses, x)
end

# Struct Generadores
# struct Gen
#     (1)Id:: Int64              
#     (2)Name::String
#     (3)Bus::Int64    
#     (4)PotMax::Float64
#     (5)PotMin::Float64
#     (6)Qmax::Float64
#     (7)Qmin::Float64
#     (8)Ramp::Int64
#     (9)Sramp::Int64
#     (10)MinUp::Int64
#     (11)MinDn::Int64
#     (12)InitS::Int64
#     (13)InitP::Int64
#     (14)StartCost::Int64
#     (15)FixedCost::Int64
#     (16)VariableCost::Int64
#     (17)Type::String
#     (18)PminFactor::Float64 
#     (19)Qfactor::Float64
#     (20)RampFactor::Float64
#     (21)StartUpCostFactor::Int64
# end

for i in 1:I
    println("Generador", i)
    x = Generador(
        generators[i,1],    # 1- Name
        generators[i,2],    # 2- Bus
        generators[i,3],    # 3- Pmax
        generators[i,4],    # 4- Pmin
        generators[i,5],    # 5- Qmax
        generators[i,6],    # 6- Qmin
        generators[i,7],    # 7- Ramp [Mw/h]
        generators[i,8],    # 8- Sramp [Mw]
        generators[i,9],    # 9- MinUp
        generators[i,10],   # 10- MinDn
        generators[i,11],   # 11- InitS
        generators[i,12],   # 12- InitP
        generators[i,13],   # 13- StartCost
        generators[i,14],   # 14- FixedCost
        generators[i,15],   # 15- VariableCost
        generators[i,16],   # 16- Type
        generators[i,17],   # 17- PminFactor
        generators[i,18],   # 18- Qfactor
        generators[i,19],   # 19- RampFactor
        generators[i,20])   # 20- StartUpCostFactor
    push!(Gen, x)
end
## println(Generadores[1].PotMax) ejemplo de como obtener un prámetro en particular

# Struct Linea
# struct Linea
#     Id::Int64
#     Name::String    
#     Inicio::Int64       # Barra de Inicio
#     Fin::Int64          # Barra de Fin
#     R::Float64          # Resistencia
#     X::Float64          # Reactancia
#     LineCharging::Int64 # (B)
#     PotMax::Int64       # Potencia maxima a transportar
# end
for i in 1:L
    x = Linea(
        lines[i,1], # 1- Name 
        lines[i,2], # 2- StartBus
        lines[i,3], # 3- EndBus
        lines[i,4], # 4- Resistance
        lines[i,5], # 5- Reactance
        lines[i,6], # 6- LineCharging(B)
        lines[i,7]) # 7- MaxFlow [Mw]   
    push!(Lineas, x)
end

println("Demanda nodo 14, hora 2: ", demandP[14,2])
println("Size de demandaP_ref: ", size(demandaP_ref))
println("Size de demandP: ", size(demandP))

# Struct demanda (quizas quitarlo)
# for i in 1:N
#     x = [demand[i,2],demand[i,3],demand[i,4],demand[i,5],demand[i,6],demand[i,7]]
#     push!(Demandas, x)
# end

# Crear Structs BESS
# for b in 1:B
#     x = BESS(batteries[b,1],batteries[b,2],batteries[b,3],batteries[b,4],
#             batteries[b,5],batteries[b,6],batteries[b,7])
#     push!(Baterias, x)
# end


####################################################################
#               MODELO OPTIMIZACION
####################################################################

### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables

# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
@objective(model, Min, sum(Gen[i].VariableCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

### Restricciones

# Equilibrio de Potenica (APROXIMACION: Flujo DC  --> Demanda Potencia Activa)
@constraint(model, DCPowerFlowConstraint[n in 1:N, t in 1:T], 
sum(p[i,t] for i in 1:I if Gen[i].Bus == n) - demandP[n,t]      == 
P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) for l in 1:L if Lineas[l].Inicio == n)
+ P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Fin,t] - d[Lineas[l].Inicio,t]) for l in 1:L if Lineas[l].Fin == n))

# + sum(pb[b,t] for b in 1:B if Baterias[b].Barra == n) | Al lado izquierdo de la ecuación (baterias)

#Flujo en lineas. Se considera o de origen, y d de destino
@constraint(model, LineMaxPotInicioFin[l in 1:L, t in 1:T], 1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base) 
@constraint(model, LineMinPotFinInicio[l in 1:L, t in 1:T], -1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base)
#Angulo de referencia
@constraint(model, RefDeg[1, t in 1:T], d[1,t] == 0)  

### Restricciones de generadores

## Potencias Activas y reactivas
# Potencia Activa maxima
@constraint(model, PMaxConstraint[i in 1:I, t in 1:T], p[i,t] <= w[i,t]*Gen[i].PotMax)
# Potencia Activa minima
@constraint(model, PMinConstraint[i in 1:I, t in 1:T], w[i,t]*Gen[i].PotMin <= p[i,t])
# Potencia Renewables (condicionado a meteorologia)
@constraint(model, RenewableMax[i in (I-R+1):I, t in 1:T], p[i,t] <= renewables[i-(I-R),t+1])

## Rampas
# Rampa up
@constraint(model, RampUpConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) <= Gen[i].Ramp + Gen[i].Sramp*u[i,t])
# Rampa dn
@constraint(model, RampDownConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) >= 0-Gen[i].Ramp - Gen[i].Sramp*v[i,t])

## Estados Binarios
@constraint(model, BinaryState[i in 1:I, t in 2:T], (u[i,t] - v[i,t]) == (w[i,t] - w[i,t-1]))
#@constraint(model, BinaryInitial[i in 1:I], (w[i,1] = State0[i]))   # Condicion inicial, por ahora libre

## Tiempo minimo de encendido: sumo todos los x dentro de la ventana desde t=1 hasta el instante enterior al encendido
#@constraint(model, MinUpTime[i in 1:I, t in 2:T], sum(w[i,k] for k in 1:(t-1) if k >= t-Gen[i].MinUp) >= v[i,t]*Gen[i].MinUp)
## Tiempo minimo de apagado
#@constraint(model, MinDnTime[i in 1:I, t in 2:T], sum((1-w[i,k]) for k in 1:(t-1) if k >= t-Gen[i].MinDn) >= u[i,t]*Gen[i].MinDn)



# Restricciones de BESS
# Energia maxima
#@constraint(model, EnergyMax[b in 1:B, t in 1:T], e[b,t] <= (Baterias[b].PotMax)*(Baterias[b].Duration))
# Potencia maxima descarga
#@constraint(model, PMaxBessDischarge[b in 1:B, t in 1:T], pb[b,t] <= Baterias[b].PotMax)
# Potencia maxima carga
#@constraint(model, PMaxBessCharge[b in 1:B, t in 1:T], pb[b,t] >= -1*(Baterias[b].PotMax))
## Imposibilidad de generacion - E inicial
#@constraint(model, Estart[b in 1:B], e[b,1] == (Baterias[b].Estart)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Imposibilidad de generacion - E final
#@constraint(model, Efinal[b in 1:B], e[b,T] == (Baterias[b].Efinal)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Dinamica BESS
#@constraint(model, DynamicBESS[b in 1:B, t in 2:T], e[b,t] == e[b,t-1] - pb[b,t])

#############################################################


################################################ ACTUALIZADO 24-05 (00:10 AM) -SEA ################################
### Notas:
#   - Siento que hay que agregar BinaryState0 -> Con los resultados se pueden analizar incongruencias
#   - Por lo mismo, las constraints de MinUpTime y MinDnTime no me terminan de convencer
#   - SI O SI, hay que fijar p[r,t] == gr[t], es decir, las potencias generadas por renovables estan
#       fijas. HINT: los renovables están al final de la tabla de generadores. 

# constraint[i in (I-a):I, t in 1:T], p[i,t] == renewable[i,t] | una wea asi

##############################################################


JuMP.optimize!(model)

#    COSTOS

totalCost = objective_value(model)
fixCost = zeros(Float64, (I,T))     # fijo
varCost = zeros(Float64, (I,T))     # variable
startCost = zeros(Float64, (I,T))   # encendido

# GUARDAR ARRAYS
for i = 1:I                                                 # Recorrer filas    (generadores)
    for t = 1:T                                             # Recorrer columnas (horas)
        fixCost[i,t] = value(w[i,t])*Gen[i].FixedCost
        varCost[i,t] = value(p[i,t])*Gen[i].VariableCost
        startCost[i,t] = value(u[i,t])*Gen[i].StartCost
    end
end
# Crear array para totales por hora
fixCostT = zeros(Float64, (1,T))
varCostT = zeros(Float64, (1,T))
startCostT = zeros(Float64, (1,T))
# Llenar array con la suma de todos los generadores en t
for t=1:T
    fixCostT[1,t] = sum(fixCost[:,t])
    varCostT[1,t] = sum(varCost[:,t])
    startCostT[1,t] = sum(startCost[:,t])
end

# CAPACIDADES NOMINALES
nomCap = zeros(Float64, (I,T))
for t=1:T
    for i=1:(I-R)   # Generadores convencionales
        nomCap[i,t] = value(w[i,t])*Gen[i].PotMax
    end
    for i=(I-R+1):I # Generadores ERNC
        nomCap[i,t] = value(w[i,t])*renewables[i-(I-R),t+1]
    end
end

# GUARDAR resultados
v_p = JuMP.value.(p)
v_u = JuMP.value.(u)
v_v = JuMP.value.(v)
v_w = JuMP.value.(w)

# EXPORTAR EXCEL
XLSX.openxlsx("ResultadosCASE14relajado.xlsx", mode="w") do xf
    
    # Cambiar nombre primera hoja ("Sheet1")
    sheet1 = xf[1]
    println(sheet1)
    XLSX.rename!(sheet1, "Potencias")

    # Agregar Hojas
    XLSX.addsheet!(xf, "Encendido")
    XLSX.addsheet!(xf, "Apagado")
    XLSX.addsheet!(xf, "State")
    XLSX.addsheet!(xf, "Costos")
    XLSX.addsheet!(xf, "PotNominal")

    # Asignar variables a cada hoja
    sheet1 = xf[1]   # Hoja 1 -> Potencias
    sheet2 = xf[2]   # Hoja 2 -> Endendidos
    sheet3 = xf[3]   # Hoja 3 -> Apagados
    sheet4 = xf[4]   # Hoja 4 -> State On/Off
    sheet5 = xf[5]   # Hoja 5 -> Costos
    sheet6 = xf[6]   # Hoja 6 -> Potencial de generacion

    # Añadir las matrices
    sheet1["A1"] = v_p
    sheet2["A1"] = v_u
    sheet3["A1"] = v_v
    sheet4["A1"] = v_w
    # Hoja costos: meter 3 vectores
    sheet5["A1"] = "Costo Encendido"
    sheet5["B1"] = startCostT
    sheet5["A2"] = "Costo Fijo"
    sheet5["B2"] = fixCostT
    sheet5["A3"] = "Costo Variable"
    sheet5["B3"] = varCostT
    # Potencial
    sheet6["A1"] = nomCap
end

Prueba de numero buses, deberia ser 14: 14
Generador1
Generador2
Generador3
Generador4
Generador5
Generador6
Generador7
Demanda nodo 14, hora 2: 12.150115830115833
Size de demandaP_ref: (14, 25)
Size de demandP: (14, 24)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26


┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers C:\Users\sevil\.julia\packages\JuMP\OUdu2\src\Containers\DenseAxisArray.jl:186


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2187 rows, 1008 columns and 5690 nonzeros
Model fingerprint: 0x534aad59
Variable types: 504 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [2e+01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 4e+02]
Presolve removed 1482 rows and 415 columns
Presolve time: 0.02s
Presolved: 705 rows, 593 columns, 2485 nonzeros
Variable types: 243 continuous, 350 integer (350 binary)
Found heuristic solution: objective 253744.55219

Root relaxation: objective 2.058054e+05, 427 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

7×24 Matrix{Float64}:
 100.0    100.0     100.0     100.0     …  100.0    100.0     100.0
  75.0     75.0      75.0      75.0         75.0     75.0      75.0
  75.0     75.0       0.0       0.0         75.0     75.0      75.0
   0.0      0.0       0.0       0.0         75.0      0.0       0.0
   0.0      0.0       0.0       0.0          0.0      0.0       0.0
  31.167   29.4256   27.6723   28.6323  …   35.471   32.5038   31.2067
   0.0      0.0       0.0       0.0          0.0      0.0       0.0

In [137]:
objective_value(model)

210536.2274729847

# Modelo 2085: case118.xlsx

In [1]:
# Tarea 2 Operacion Economica de Sistemas Electricos
# Vicente Goehring & Sebastian Villagra
# Mayo 2024

# Codigo busca crear un modelo de optimizacion para el Unit Commitment
# de forma mas general posible, i.e., entra como input cualquier CSV con
# los datos de un sistema y optimiza de forma robusta.


#Iniciación de paquetes
using DataFrames, CSV
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi
using XLSX  # Se agrega para leer archivos .xlsx (Excel)

#Definición de struct: FALTA AGREGAR MAS INSTANCIAS DE CADA STRUCT (seba)  

# Barras
struct Bus
    Id::Int64       # Id del Bus
    Vmax::Float64   # Voltaje maximo [pu]
    Vmin::Float64   # Voltaje minimo [pu]
    Gs::Float64
    Bs::Float64
end

# Generadores
struct Generador
    Name::String
    Bus::Int64    
    PotMax::Float64
    PotMin::Float64
    Qmax::Float64
    Qmin::Float64
    Ramp::Float64
    Sramp::Float64
    MinUp::Float64
    MinDn::Float64
    InitS::Float64
    InitP::Float64
    StartCost::Float64
    FixedCost::Float64
    VariableCost::Float64
    Tech::String
    PminFactor::Float64 
    Qfactor::Float64
    RampFactor::Float64
    StartUpCostFactor::Int64
end

# Potencias demandadas
# La demanda la dejaria como un DataFrame y listo
struct Demanda
    T::Int64
    Barra1::Int64
    Barra2::Int64
    Barra3::Int64
    Barra4::Int64
    Barra5::Int64
    Barra6::Int64
    Barra7::Int64
    Barra8::Int64
    Barra9::Int64
end

# Lineas de transmision
# IMPORTANTE:   case014.xlsx tiene el inicio y fin como string. case118.xlsx los tiene como int
#               Por lo tanto, se modifica el archivo case014.xlsx para poder leer de forma automatica
struct Linea
    Name::String    
    Inicio::Int64       # Barra de Inicio
    Fin::Int64          # Barra de Fin
    R::Float64          # Resistencia
    X::Float64          # Reactancia
    LineCharging::Int64 # (B)
    PotMax::Int64       # Potencia maxima a transportar
end
# Battery Energy Storage System
struct BESS
    Id::Int64
    PotMax::Int64
    Duration::Int64
    Eff::Float64
    Estart::Float64
    Efinal::Float64
    Barra::Int64
end
# Renewables -> tambien dejarlo como DataFrame

##  Archivo Excel a leer
archivo = "Case118.xlsx"    # se comienza con el caso mas pequeño
xf = XLSX.readxlsx(archivo) # leer el archivo excel

# Separar hojas del archivo
sheet_buses = xf["Buses"]
sheet_demandas = xf["Demand"]
sheet_generadores = xf["Generators"]
sheet_lineas = xf["Lines"]
sheet_renovables = xf["Renewables"]

# Definir la función de parada
# Se crea el DataFrame hasta que se reconozca la entrada "END"
# De esta forma se puede leer tablas con cualquier numero de filas
#   siempre y cuando terminen con un "END".
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

# Crear DataFrames
# XLSX.gettable(objeto hoja excel, Columnas de tabla, first_row= primera fila a considerar, stop_in_row_function: condicion dejar de leer)
buses_ref       = DataFrame(XLSX.gettable(sheet_buses,"A:E",first_row=1,stop_in_row_function=stop_condition))
demandaP_ref    = DataFrame(XLSX.gettable(sheet_demandas,"A:Y",first_row=2,stop_in_row_function=stop_condition))
demandaQ_ref    = DataFrame(XLSX.gettable(sheet_demandas,"AA:AY",first_row=2,stop_in_row_function=stop_condition))
generadores_ref = DataFrame(XLSX.gettable(sheet_generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))
lineas_ref      = DataFrame(XLSX.gettable(sheet_lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))
renovables_ref  = DataFrame(XLSX.gettable(sheet_renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))
# Notar que se crean 2 DataFrame a partir de la hoja "Demand": Potencia activa (P) y potencia reactiva (Q).
# Ademas, la fila considerada como inicio de la tabla es la fila 2, ya que la fila 1 solo hace distincion entre estas 2 potencias.
# Lo mismo ocurre en el DataFrame "renovables_ref".

# ESTO SE ELIMINARIA
#######################################################
#Importación de archivos
#generators_ref  = CSV.File("Generators.csv") |> DataFrame   # Crear DataFrame generadores
#demand_ref      = CSV.File("Demand.csv") |> DataFrame       # Crear DataFrame demanda
#lines_ref       = CSV.File("Lines.csv") |> DataFrame        # Crear DataFrame lineas tx
#batteries_ref   = CSV.File("Bess.csv") |> DataFrame         # Crear DataFrame baterias
# Crear una copia 
#generators      = copy(generators_ref)
#demand          = copy(demand_ref)
#lines           = copy(lines_ref)
#batteries       = copy(batteries_ref)
#######################################################

# Crear copias (aparentemente buena practica)
buses = copy(buses_ref)
demandP = copy(demandaP_ref[:,2:25])
demandQ = copy(demandaQ_ref)
generators = copy(generadores_ref)
lines = copy(lineas_ref)
renewables = copy(renovables_ref) 

#inclusión en variables
P_base = 100
T = ncol(demandP)           # N° bloques temporales
N = nrow(buses)             # N° nodos 
I = nrow(generators)        # N° Generadores
L = nrow(lines)             # N° Lineas
R = nrow(renewables)        # N° Renovables
#B = nrow(batteries)        # N° BESS

# Debugg check
println("Prueba de numero buses, deberia ser 14: ", N)

# HASTA ACA LLEGUE (SEA -quien mas xdxdxdxd): 17/05 -

# Listas que almacenan los Structs 
Buses       = []
Gen         = []
Demandas    = []
Lineas      = []
#Baterias    = []

## Crear los Structs

# Struct Bus
# struct Bus
#     Id::Int64       # Id del Bus
#     Vmax::Float64   # Voltaje maximo [pu]
#     Vmin::Float64   # Voltaje minimo [pu]
#     Gs::Float64
#     Bs::Float64
# end
for i in 1:N
    x = Bus(
        buses[i,1], # 1- Id
        buses[i,2], # 2- Vmax [pu]
        buses[i,3], # 3- Vmin [pu]
        buses[i,4], # 4- Gs
        buses[i,5]) # 8- Bs   
    push!(Buses, x)
end

# Struct Generadores
# struct Gen
#     (1)Id:: Int64              
#     (2)Name::String
#     (3)Bus::Int64    
#     (4)PotMax::Float64
#     (5)PotMin::Float64
#     (6)Qmax::Float64
#     (7)Qmin::Float64
#     (8)Ramp::Int64
#     (9)Sramp::Int64
#     (10)MinUp::Int64
#     (11)MinDn::Int64
#     (12)InitS::Int64
#     (13)InitP::Int64
#     (14)StartCost::Int64
#     (15)FixedCost::Int64
#     (16)VariableCost::Int64
#     (17)Type::String
#     (18)PminFactor::Float64 
#     (19)Qfactor::Float64
#     (20)RampFactor::Float64
#     (21)StartUpCostFactor::Int64
# end

for i in 1:I
    println("Generador", i)
    x = Generador(
        generators[i,1],    # 1- Name
        generators[i,2],    # 2- Bus
        generators[i,3],    # 3- Pmax
        generators[i,4],    # 4- Pmin
        generators[i,5],    # 5- Qmax
        generators[i,6],    # 6- Qmin
        generators[i,7],    # 7- Ramp [Mw/h]
        generators[i,8],    # 8- Sramp [Mw]
        generators[i,9],    # 9- MinUp
        generators[i,10],   # 10- MinDn
        generators[i,11],   # 11- InitS
        generators[i,12],   # 12- InitP
        generators[i,13],   # 13- StartCost
        generators[i,14],   # 14- FixedCost
        generators[i,15],   # 15- VariableCost
        generators[i,16],   # 16- Type
        generators[i,17],   # 17- PminFactor
        generators[i,18],   # 18- Qfactor
        generators[i,19],   # 19- RampFactor
        generators[i,20])   # 20- StartUpCostFactor
    push!(Gen, x)
end
## println(Generadores[1].PotMax) ejemplo de como obtener un prámetro en particular

# Struct Linea
# struct Linea
#     Id::Int64
#     Name::String    
#     Inicio::Int64       # Barra de Inicio
#     Fin::Int64          # Barra de Fin
#     R::Float64          # Resistencia
#     X::Float64          # Reactancia
#     LineCharging::Int64 # (B)
#     PotMax::Int64       # Potencia maxima a transportar
# end
for i in 1:L
    x = Linea(
        lines[i,1], # 1- Name 
        lines[i,2], # 2- StartBus
        lines[i,3], # 3- EndBus
        lines[i,4], # 4- Resistance
        lines[i,5], # 5- Reactance
        lines[i,6], # 6- LineCharging(B)
        lines[i,7]) # 7- MaxFlow [Mw]   
    push!(Lineas, x)
end

println("Demanda nodo 14, hora 2: ", demandP[14,2])
println("Size de demandaP_ref: ", size(demandaP_ref))
println("Size de demandP: ", size(demandP))

# Struct demanda (quizas quitarlo)
# for i in 1:N
#     x = [demand[i,2],demand[i,3],demand[i,4],demand[i,5],demand[i,6],demand[i,7]]
#     push!(Demandas, x)
# end

# Crear Structs BESS
# for b in 1:B
#     x = BESS(batteries[b,1],batteries[b,2],batteries[b,3],batteries[b,4],
#             batteries[b,5],batteries[b,6],batteries[b,7])
#     push!(Baterias, x)
# end


####################################################################
#               MODELO OPTIMIZACION
####################################################################

### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables

# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
@objective(model, Min, sum(Gen[i].VariableCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

### Restricciones

# Equilibrio de Potenica (APROXIMACION: Flujo DC  --> Demanda Potencia Activa)
@constraint(model, DCPowerFlowConstraint[n in 1:N, t in 1:T], 
sum(p[i,t] for i in 1:I if Gen[i].Bus == n) - demandP[n,t]      == 
P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) for l in 1:L if Lineas[l].Inicio == n)
+ P_base*sum( (1/Lineas[l].X) * (d[Lineas[l].Fin,t] - d[Lineas[l].Inicio,t]) for l in 1:L if Lineas[l].Fin == n))

# + sum(pb[b,t] for b in 1:B if Baterias[b].Barra == n) | Al lado izquierdo de la ecuación (baterias)

#Flujo en lineas. Se considera o de origen, y d de destino
@constraint(model, LineMaxPotInicioFin[l in 1:L, t in 1:T], 1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base) 
@constraint(model, LineMinPotFinInicio[l in 1:L, t in 1:T], -1/Lineas[l].X * (d[Lineas[l].Inicio,t] - d[Lineas[l].Fin,t]) <= Lineas[l].PotMax/P_base)
#Angulo de referencia
@constraint(model, RefDeg[1, t in 1:T], d[1,t] == 0)  

### Restricciones de generadores

## Potencias Activas y reactivas
# Potencia Activa maxima
@constraint(model, PMaxConstraint[i in 1:I, t in 1:T], p[i,t] <= w[i,t]*Gen[i].PotMax)
# Potencia Activa minima
@constraint(model, PMinConstraint[i in 1:I, t in 1:T], w[i,t]*Gen[i].PotMin <= p[i,t])
# Potencia Renewables (condicionado a meteorologia)
@constraint(model, RenewableMax[i in (I-R+1):I, t in 1:T], p[i,t] <= renewables[i-(I-R),t+1])

## Rampas
# Rampa up
@constraint(model, RampUpConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) <= Gen[i].Ramp + Gen[i].Sramp*u[i,t])
# Rampa dn
@constraint(model, RampDownConstaint[i in 1:I, t in 2:T], (p[i,t] - p[i,t-1]) >= 0-Gen[i].Ramp - Gen[i].Sramp*v[i,t])

## Estados Binarios
@constraint(model, BinaryState[i in 1:I, t in 2:T], (u[i,t] - v[i,t]) == (w[i,t] - w[i,t-1]))
#@constraint(model, BinaryInitial[i in 1:I], (w[i,1] = State0[i]))   # Condicion inicial, por ahora libre

## Tiempo minimo de encendido: sumo todos los x dentro de la ventana desde t=1 hasta el instante enterior al encendido
@constraint(model, MinUpTime[i in 1:I, t in 2:T], sum(w[i,k] for k in 1:(t-1) if k >= t-Gen[i].MinUp) >= v[i,t]*Gen[i].MinUp)
## Tiempo minimo de apagado
@constraint(model, MinDnTime[i in 1:I, t in 2:T], sum((1-w[i,k]) for k in 1:(t-1) if k >= t-Gen[i].MinDn) >= u[i,t]*Gen[i].MinDn)



# Restricciones de BESS
# Energia maxima
#@constraint(model, EnergyMax[b in 1:B, t in 1:T], e[b,t] <= (Baterias[b].PotMax)*(Baterias[b].Duration))
# Potencia maxima descarga
#@constraint(model, PMaxBessDischarge[b in 1:B, t in 1:T], pb[b,t] <= Baterias[b].PotMax)
# Potencia maxima carga
#@constraint(model, PMaxBessCharge[b in 1:B, t in 1:T], pb[b,t] >= -1*(Baterias[b].PotMax))
## Imposibilidad de generacion - E inicial
#@constraint(model, Estart[b in 1:B], e[b,1] == (Baterias[b].Estart)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Imposibilidad de generacion - E final
#@constraint(model, Efinal[b in 1:B], e[b,T] == (Baterias[b].Efinal)*(Baterias[b].PotMax)*(Baterias[b].Duration))
# Dinamica BESS
#@constraint(model, DynamicBESS[b in 1:B, t in 2:T], e[b,t] == e[b,t-1] - pb[b,t])

#############################################################


################################################ ACTUALIZADO 24-05 (00:10 AM) -SEA ################################
### Notas:
#   - Siento que hay que agregar BinaryState0 -> Con los resultados se pueden analizar incongruencias
#   - Por lo mismo, las constraints de MinUpTime y MinDnTime no me terminan de convencer
#   - SI O SI, hay que fijar p[r,t] == gr[t], es decir, las potencias generadas por renovables estan
#       fijas. HINT: los renovables están al final de la tabla de generadores. 

# constraint[i in (I-a):I, t in 1:T], p[i,t] == renewable[i,t] | una wea asi

##############################################################


JuMP.optimize!(model)

#    COSTOS

totalCost = objective_value(model)
fixCost = zeros(Float64, (I,T))     # fijo
varCost = zeros(Float64, (I,T))     # variable
startCost = zeros(Float64, (I,T))   # encendido

# GUARDAR ARRAYS
for i = 1:I                                                 # Recorrer filas    (generadores)
    for t = 1:T                                             # Recorrer columnas (horas)
        fixCost[i,t] = value(w[i,t])*Gen[i].FixedCost
        varCost[i,t] = value(p[i,t])*Gen[i].VariableCost
        startCost[i,t] = value(u[i,t])*Gen[i].StartCost
    end
end
# Crear array para totales por hora
fixCostT = zeros(Float64, (1,T))
varCostT = zeros(Float64, (1,T))
startCostT = zeros(Float64, (1,T))
# Llenar array con la suma de todos los generadores en t
for t=1:T
    fixCostT[1,t] = sum(fixCost[:,t])
    varCostT[1,t] = sum(varCost[:,t])
    startCostT[1,t] = sum(startCost[:,t])
end

# CAPACIDADES NOMINALES
nomCap = zeros(Float64, (I,T))
for t=1:T
    for i=1:(I-R)   # Generadores convencionales
        nomCap[i,t] = value(w[i,t])*Gen[i].PotMax
    end
    for i=(I-R+1):I # Generadores ERNC
        nomCap[i,t] = value(w[i,t])*renewables[i-(I-R),t+1]
    end
end

# GUARDAR resultados
v_p = JuMP.value.(p)
v_u = JuMP.value.(u)
v_v = JuMP.value.(v)
v_w = JuMP.value.(w)

# EXPORTAR EXCEL
XLSX.openxlsx("ResultadosCASE118.xlsx", mode="w") do xf
    
    # Cambiar nombre primera hoja ("Sheet1")
    sheet1 = xf[1]
    println(sheet1)
    XLSX.rename!(sheet1, "Potencias")

    # Agregar Hojas
    XLSX.addsheet!(xf, "Encendido")
    XLSX.addsheet!(xf, "Apagado")
    XLSX.addsheet!(xf, "State")
    XLSX.addsheet!(xf, "Costos")
    XLSX.addsheet!(xf, "PotNominal")

    # Asignar variables a cada hoja
    sheet1 = xf[1]   # Hoja 1 -> Potencias
    sheet2 = xf[2]   # Hoja 2 -> Endendidos
    sheet3 = xf[3]   # Hoja 3 -> Apagados
    sheet4 = xf[4]   # Hoja 4 -> State On/Off
    sheet5 = xf[5]   # Hoja 5 -> Costos
    sheet6 = xf[6]   # Hoja 6 -> Potencial de generacion

    # Añadir las matrices
    sheet1["A1"] = v_p
    sheet2["A1"] = v_u
    sheet3["A1"] = v_v
    sheet4["A1"] = v_w
    # Hoja costos: meter 3 vectores
    sheet5["A1"] = "Costo Encendido"
    sheet5["B1"] = startCostT
    sheet5["A2"] = "Costo Fijo"
    sheet5["B2"] = fixCostT
    sheet5["A3"] = "Costo Variable"
    sheet5["B3"] = varCostT
    # Potencial
    sheet6["A1"] = nomCap
end

Prueba de numero buses, deberia ser 14: 118
Generador1
Generador2
Generador3
Generador4
Generador5
Generador6
Generador7
Generador8
Generador9
Generador10
Generador11
Generador12
Generador13
Generador14
Generador15
Generador16
Generador17
Generador18
Generador19
Generador20
Generador21
Generador22
Generador23
Generador24
Generador25
Generador26
Generador27
Generador28
Generador29
Generador30
Generador31
Generador32
Generador33
Generador34
Generador35
Generador36
Generador37
Generador38
Generador39
Generador40
Generador41
Generador42
Generador43
Generador44
Generador45
Generador46
Generador47
Generador48
Generador49
Generador50
Generador51
Generador52
Generador53
Generador54
Generador55
Generador56
Generador57
Generador58
Generador59
Generador60
Generador61
Generador62
Generador63
Generador64
Generador65
Generador66
Generador67
Generador68
Generador69
Generador70
Generador71
Generador72
Generador73
Generador74
Generador75
Generador76
Generador77
Generador78
Generador79
Generador80
Gener

┌ Warning: Axis contains one element: 1. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[1]` instead of `1`.
└ @ JuMP.Containers C:\Users\vicho\.julia\packages\JuMP\OUdu2\src\Containers\DenseAxisArray.jl:186


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 31806 rows, 13776 columns and 87004 nonzeros
Model fingerprint: 0xefb6df4f
Variable types: 5568 continuous, 8208 integer (8208 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [8e+00, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 5e+02]
Presolve removed 9880 rows and 3106 columns
Presolve time: 0.41s
Presolved: 21926 rows, 10670 columns, 66968 nonzeros
Variable types: 5114 continuous, 5556 integer (5556 binary)

Root relaxation: objective 3.433998e+06, 8531 iterations, 0.76 seconds (0.79 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 34339

114×24 Matrix{Float64}:
 100.0  100.0  100.0  100.0  …  100.0  100.0  100.0  100.0  100.0
 491.0  491.0  491.0  491.0     491.0  491.0  491.0  491.0  491.0
 492.0  492.0  492.0  492.0     492.0  492.0  492.0  492.0  492.0
 320.0  320.0  320.0  320.0     320.0  320.0  320.0  320.0  320.0
 104.0  104.0  104.0  104.0     104.0  104.0  104.0  104.0  104.0
 707.0  707.0  707.0  707.0  …  707.0  707.0  707.0  707.0  707.0
 119.0  119.0  119.0  119.0     119.0  119.0  119.0  119.0  119.0
 304.0  304.0  304.0  304.0     304.0  304.0  304.0  304.0  304.0
 100.0  100.0  100.0  100.0     100.0  100.0  100.0  100.0  100.0
 148.0  148.0  148.0  148.0     148.0  148.0  148.0  148.0  148.0
   ⋮                         ⋱           ⋮                  
   0.0    0.0    0.0    0.0  …    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.

In [2]:
solution_summary(model)

* Solver : Gurobi

* Status
  Result count       : 10
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.49422e+06
  Objective bound    : 3.49403e+06
  Relative gap       : 5.39389e-05
  Dual objective value : 3.49403e+06

* Work counters
  Solve time (sec)   : 3.38620e+01
  Barrier iterations : 0
  Node count         : 4722
